In [1]:
import pandas as pd 
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score


In [2]:
df = pd.read_csv(r'..//data/processed/df_limpo.csv')

In [31]:
df = df[:10000]

In [ ]:
freq   = CountVectorizer()
corpus = freq.fit_transform(df['body'])

onehot = Binarizer()
corpus = onehot.fit_transform(corpus.toarray())


In [ ]:
import collections, numpy
a = numpy.array(corpus[1000])
counter = collections.Counter(a)

print(counter)

In [32]:
le = LabelEncoder()
y = le.fit_transform(df['tags'])

In [ ]:
le.class_

In [33]:
x_train, x_test, y_train, y_test = train_test_split(df['body'], y, test_size=0.2, random_state=5)

In [34]:
pipeline = Pipeline([
    ('vectorizer',TfidfVectorizer(binary=True, use_idf=False, norm=False)),
    ('clf', MLPClassifier())
])

In [35]:
pipeline.fit(x_train,y_train)

In [25]:
pre = pipeline.predict(x_test)

In [26]:
accuracy_score(y_test,pre)

0.267